In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, time, os, math
import tmdbsimple as tmdb
from tqdm.notebook import tqdm
tmdb.API_KEY =  '56a16fe8a7dd1bf1c8e388a0c11ab94f'
#For our project we will need to extract 3 pieces of information for each movie:Revenue, Budget, Certification

In [2]:
#save loc for api call data
FOLDER = "Loading_Cleaning/Data"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'Datafinal_tmdb_data_2000.csv.gz',
 'Datafinal_tmdb_data_2001.csv.gz',
 'Datatmdb_api_results_2000.json',
 'Datatmdb_api_results_2001.json',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [3]:
basics = pd.read_csv(r'Loading_Cleaning/Data/title_basics.csv.gz', low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy


In [4]:
def movie_with_rating(movie_id):
# Get movie and release dates
    movie = tmdb.Movies(movie_id)
# construct output dict
    info = movie.info()
    releases = movie.releases()
# Loop through countries in releases
    for c in releases['countries']:
    # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
    return info

In [5]:
def write_json(new_data, filename): 
        """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
        with open(filename,'r+') as file:
            file_data = json.load(file)
            if (type(new_data) == list) & (type(file_data) == list):
                file_data.extend(new_data)
            else:
                 file_data.append(new_data)
        
            file.seek(0)
            json.dump(file_data, file)

In [6]:
#years I am seeking to query
YEARS_TO_GET = [2000,2001]

In [7]:
for YEAR in tqdm(YEARS_TO_GET,desc='YEARS',position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)
    df = basics.loc[ basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    for movie_id in tqdm(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
            try:
                temp = movie_with_rating(movie_id)  
                write_json(temp,JSON_FILE)
                time.sleep(0.02)
            except Exception as e:
                continue
            
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1408 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1526 [00:00<?, ?it/s]